# Add sentence reference to each sentence in every story for each word in common words

Ref: storynumber_sentencenumber

In [1]:
import pandas as pd
import os
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\haafla\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\haafla\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [4]:
import os
import re
import pandas as pd
from nltk.tokenize import sent_tokenize

# Open common_words.csv
common_words = pd.read_csv('./data/common_words.csv', delimiter=';')

# Empty all sentence references for each word before processing stories
common_words['Sentence references'] = '[]'

# Process each story file in the stories folder
for file in os.listdir('./stories'):
  story_num = file.split('_')[0]
  with open(os.path.join('./stories', file), 'r') as f:
    story = f.read()
  
  # Clean the story text
  story = story.replace('\n', ' ').replace('”', '').replace('“', '')
  
  # Use only the Italian story part (if applicable)
  it_story = story.split(';;;')[0]
  
  # Split the story into sentences
  sentences = sent_tokenize(it_story)
  
  # For each common word, update its sentence references
  for index, row in common_words.iterrows():
    word = row['Word']
    
    # Retrieve the current sentence references (if any)
    current_refs = []
    cell = row.get('Sentence references', '')
    if pd.notna(cell) and cell.strip():
      try:
        current_refs = eval(cell)  # assuming the cell is a string like "['1_0', '2_3']"
      except Exception as e:
        current_refs = []
    
    # Check each sentence for the word and add new references
    for i, sentence in enumerate(sentences):
      # Find all words in the sentence
      words_in_sentence = re.findall(r'\b\w+\b', sentence)
      if word in words_in_sentence:
        new_ref = f'{story_num}_{i}'
        if new_ref not in current_refs:
          current_refs.append(new_ref)
    
    # Update the DataFrame cell with the new references (stored as a string)
    common_words.at[index, 'Sentence references'] = str(current_refs)
  
  # Save the updated common_words DataFrame to CSV after each story
  common_words.to_csv('./data/common_words.csv', sep=';', index=False)


In [5]:
# Check how many words in the common_words.csv have sentence references
common_words = pd.read_csv('./data/common_words.csv', delimiter=';')
# Turn sentence ref from string to list ("[1, 2, 3]" -> [1, 2, 3])
common_words['Sentence references'] = common_words['Sentence references'].apply(lambda x: eval(x))
# Create value counts of length of sentence references
sentence_ref_counts = common_words['Sentence references'].apply(lambda x: len(x)).value_counts()
# Print sorted value counts
print(sentence_ref_counts.sort_index())

Sentence references
0     804
1      84
2      36
3      22
4      12
5       9
6       4
7       2
8       6
9       1
10      3
13      2
16      1
17      1
21      2
25      2
26      2
33      2
47      3
60      1
62      1
Name: count, dtype: int64


In [5]:
sentences

['C’era una volta, al centro di una fitta foresta, una casetta, dove viveva una graziosa bambina chiamata Cappuccetto Rosso.',
 'Un giorno assolato, la madre la chiamò nella cucina della casetta.',
 '«La nonna è malata,» disse.',
 '«Portale questa cesta di focaccine, ma stai attenta!',
 'Resta sul sentiero quando attraversi la foresta e non fermarti!',
 'Se cammini in fretta, non incontrerai pericoli.»  «Non preoccuparti,» disse, «correrò dalla nonna senza fermarmi.» Cappuccetto Rosso baciò la madre e se ne andò.',
 'La bambina iniziò il lungo tragitto nella foresta.',
 '...Ma ben presto dimenticò le parole sagge della madre.',
 'Vide un cespuglio di more accanto al sentiero.',
 '«Che belle more!» esclamò, poggiando a terra il cestino.',
 '«Sono così graziose e mature... e così grandi!',
 'Deliziose!',
 'Forse potrei provarne ancora una...»  Improvvisamente, Cappuccetto Rosso ricordò la madre, la sua promessa, la nonna e il cestino... Ritornò in fretta sul sentiero.',
 'La foresta dive